# Deeper Depth Prediction with Fully Convolutional Residual Networks

By [Iro Laina](http://campar.in.tum.de/Main/IroLaina), [Christian Rupprecht](http://campar.in.tum.de/Main/ChristianRupprecht), [Vasileios Belagiannis](http://www.robots.ox.ac.uk/~vb/), [Federico Tombari](http://campar.in.tum.de/Main/FedericoTombari), [Nassir Navab](http://campar.in.tum.de/Main/NassirNavab).

Modified from predict.py

In [8]:
import argparse
import glob, os
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from matplotlib import image as im
%matplotlib inline
from PIL import Image
import time

import models

## Loading the model

In [9]:
# Path to pre-trained model
model_data_path = "./NYU_ResNet-UpProj.npy"
# Default input size
height = 228
width = 304
channels = 3
batch_size = 1
# Create a placeholder for the input image
input_node = tf.placeholder(tf.float32, shape=(None, height, width, channels))
    
# Construct the network
net = models.ResNet50UpProj({'data': input_node}, batch_size)
    
sess = tf.Session()
    
# Load the converted parameters
start_time = time.time()
print('Loading the model...')
net.load(model_data_path, sess)
    
uninitialized_vars = []
for var in tf.global_variables():
    try:
        sess.run(var)
    except tf.errors.FailedPreconditionError:
        uninitialized_vars.append(var)
            
init_new_vars_op = tf.variables_initializer(uninitialized_vars)
sess.run(init_new_vars_op)
print('Model is ready')
print("--- Model loading time: %s seconds ---" % (time.time() - start_time))

Loading the model...
Model is ready
--- Model loading time: 136.79332733154297 seconds ---


## Predict the single depth images
### You can skip this step to the next one if you want to predict series of images

In [ ]:
start_time = time.time()
print('Reading image')
# Read image
image_path = "./Images/new_office.jpg"
img = Image.open(image_path)
img = img.resize([width,height], Image.ANTIALIAS)
img = np.array(img).astype('float32')
img = np.expand_dims(np.asarray(img), axis = 0)
print("--- Image reading time : %s seconds ---" % (time.time() - start_time))
print('Predicting')
start_time = time.time()
pred = sess.run(net.get_output(), feed_dict={input_node: img})
print(type(pred))
print("--- Predicting time : %s seconds ---" % (time.time() - start_time))

# Plot result
fig = plt.figure()
ii = plt.imshow(pred[0,:,:,0], interpolation='nearest', cmap=plt.cm.get_cmap('Greys_r'))
fig.colorbar(ii)
plt.show()


Reading image
--- Image reading time : 0.01200103759765625 seconds ---
Predicting


Predict multiple depth images
------------------------

In [15]:
# test image saving
im.imsave('name.jpg',pred[0,:,:,0],cmap=plt.cm.get_cmap('viridis'))

In [15]:
seriesPath = ".\series"
outputPath = ".\output"
seriesFiles = seriesPath + "\*.png"
# Length of path
seriesPathLength = len(seriesPath)
print(seriesPathLength)
start_time = time.time()
i = 0
# get file name
for file in glob.glob(seriesFiles):
    filename = file[seriesPathLength + 1:]
    print(filename)
    img = Image.open(file)
    img = img.resize([width,height], Image.ANTIALIAS)
    img = np.array(img).astype('float32')
    img = np.expand_dims(np.asarray(img), axis = 0)
    # predict
    pred = sess.run(net.get_output(), feed_dict={input_node: img})
    # save the depth map
    im.imsave(outputPath + "\\" + filename,pred[0,:,:,0],cmap=plt.cm.get_cmap('Greys'))
    print (filename + " saved" + " in" + outputPath)
    i += 1
    
    
print("--- Predicting time : %s seconds ---" % (time.time() - start_time))
    


8
1341847980.722988.png
1341847980.722988.png saved in.\output
1341847980.754743.png
1341847980.754743.png saved in.\output
1341847980.786856.png
1341847980.786856.png saved in.\output
1341847980.822978.png
1341847980.822978.png saved in.\output
1341847980.854676.png
1341847980.854676.png saved in.\output
1341847980.890728.png
1341847980.890728.png saved in.\output
1341847980.922978.png
1341847980.922978.png saved in.\output
1341847980.954645.png
1341847980.954645.png saved in.\output
1341847980.990699.png
1341847980.990699.png saved in.\output
1341847981.022715.png
1341847981.022715.png saved in.\output
1341847981.054711.png
1341847981.054711.png saved in.\output
1341847981.090715.png
1341847981.090715.png saved in.\output
1341847981.122985.png
1341847981.122985.png saved in.\output
1341847981.158632.png
1341847981.158632.png saved in.\output
1341847981.190636.png
1341847981.190636.png saved in.\output
1341847981.222978.png
1341847981.222978.png saved in.\output
1341847981.258722.png


In [ ]:
sess.close()